In [1]:
!pip install openai pandas tqdm

In [ ]:
import pandas as pd
import openai
from tqdm import tqdm
import time

# Inicializa o cliente da Maritaca
client = openai.OpenAI(
    api_key="INSERIR_CHAVE_API",
    base_url="https://chat.maritaca.ai/api"
)

# Carrega o CSV
caminho_arquivo = r'C:\Users\Documents\Dados-Sinteticos\Dados\dataset-petitions-legal-br.csv'
df = pd.read_csv(caminho_arquivo).reset_index(drop=True)

# Prompt builder
def criar_prompt_formatado(fato, categoria):
    return f"""
Você é um(a) assistente jurídico(a) especializado(a) na geração de documentos legais sintéticos e totalmente anonimizados.

Sua tarefa é criar uma petição jurídica simulada, baseada no fato apresentado abaixo, garantindo:

1. **Anonimização completa e irreversível**:
   - Substitua todos os dados pessoais por marcadores padronizados como:
     - [NOME_ANONIMIZADO]
     - [CPF_ANONIMIZADO]
     - [RG_ANONIMIZADO]
     - [DATA_ANONIMIZADA]
     - [ENDERECO_ANONIMIZADO]
     - [EMAIL_ANONIMIZADO]
     - [TELEFONE_ANONIMIZADO]
     - [CIDADE_ANONIMIZADA]
     - [ESTADO_ANONIMIZADO]
     - [NUMERO_PROCESSO_ANONIMIZADO]
     - [DEFENSORIA_ANONIMIZADA]
     - [DEFENSOR_ANONIMIZADO]
     - [FORUM_ANONIMIZADO]
     - [BANCO_ANONIMIZADO]
     - [ESCOLA_ANONIMIZADA]
     - [EMPRESA_ANONIMIZADA]
   - Isso inclui **nomes próprios, instituições, locais, datas, números de documentos, telefones, endereços, e-mails e qualquer outra informação que permita identificação direta ou indireta**.

2. **Coerência jurídica e linguagem formal**:
   - Estruture o texto como uma petição tradicional, com uso de parágrafos claros, pontuação adequada e vocabulário jurídico.
   - Mantenha o raciocínio lógico e o encadeamento textual, respeitando o estilo profissional de uma peça judicial.

3. **Aderência à categoria do caso**:
   - O conteúdo deve refletir e respeitar a natureza da seguinte categoria jurídica: **{categoria}**

---

### Fato base (original):

\"\"\"{fato}\"\"\"

---

Agora, com base nessas diretrizes, gere o texto jurídico sintético, formatado, anonimizando completamente todas as referências pessoais e institucionais.
"""

def contar_tokens(texto):
    return int(len(texto.split()) * 1.3)

# Limites da Maritaca (Tier 0)
RPM = 60
TPM_INPUT = 128000
TPM_OUTPUT = 10000
MAX_OUTPUT_TOKENS = 1500

# Estado inicial
fatos_sinteticos = []
checkpoint_path = r'C:\Users\Documents\Dados-Sinteticos\Saida_Sinteticos\checkpoint_sabia.csv'

# Tenta retomar de onde parou
try:
    df_checkpoint = pd.read_csv(checkpoint_path)
    inicio_idx = len(df_checkpoint)
    fatos_sinteticos = df_checkpoint['facts_sinteticos_formatados'].tolist()
    print(f"Retomando do índice {inicio_idx}")
except:
    inicio_idx = 0
    print("Iniciando do zero.")

tokens_usados_input = 0
tokens_usados_output = 0
reqs_no_minuto = 0
inicio_minuto = time.time()

for idx in tqdm(range(inicio_idx, len(df))):
    row = df.iloc[idx]
    prompt = criar_prompt_formatado(row['facts'], row['issue_area'])

    tokens_prompt = contar_tokens(prompt)
    tokens_resposta = MAX_OUTPUT_TOKENS
    tempo_passado = time.time() - inicio_minuto

    while (
        (tokens_usados_input + tokens_prompt > TPM_INPUT) or
        (tokens_usados_output + tokens_resposta > TPM_OUTPUT) or
        (reqs_no_minuto + 1 > RPM)
    ):
        esperar = max(0.1, 60 - tempo_passado)
        print(f"Aguardando {esperar:.1f}s para respeitar o rate limit...")
        time.sleep(esperar)
        inicio_minuto = time.time()
        tokens_usados_input = 0
        tokens_usados_output = 0
        reqs_no_minuto = 0
        tempo_passado = 0

    try:
        response = client.chat.completions.create(
            model="sabia-3.1",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=MAX_OUTPUT_TOKENS
        )

        resposta_texto = response.choices[0].message.content.strip()
        fatos_sinteticos.append(resposta_texto)
        tokens_usados_input += tokens_prompt
        tokens_usados_output += contar_tokens(resposta_texto)
        reqs_no_minuto += 1

    except Exception as e:
        print(f"Erro no item {idx}: {e}")
        fatos_sinteticos.append("ERRO")

    # Salva a cada item
    df_parcial = df.iloc[:inicio_idx + len(fatos_sinteticos)].copy()
    df_parcial['facts_sinteticos_formatados'] = fatos_sinteticos
    df_parcial.to_csv(checkpoint_path, index=False)

# Salva arquivo final
output_path = r'C:\Users\Documents\Dados-Sinteticos\Saida_Sinteticos\dataset_peticoes_sintetico_formatado.csv'
df['facts_sinteticos_formatados'] = fatos_sinteticos
df.to_csv(output_path, index=False)
print(f"Arquivo final salvo em: {output_path}")

Iniciando do zero.


  0%|          | 15/3458 [04:52<19:33:39, 20.45s/it]

Aguardando 0.1s para respeitar o rate limit...


  1%|          | 28/3458 [09:33<19:23:19, 20.35s/it]

Aguardando 0.1s para respeitar o rate limit...


...

Aguardando 0.1s para respeitar o rate limit...


100%|█████████▉| 3446/3458 [26:05:05<04:10, 20.88s/it]

Aguardando 0.1s para respeitar o rate limit...


100%|██████████| 3458/3458 [26:09:29<00:00, 27.23s/it]


Arquivo final salvo em: C:\Users\Willgnner\Documents\Dados-Sinteticos\Saida_Sinteticos\dataset_peticoes_sintetico_formatado.csv


In [ ]:
import pandas as pd

# Caminho do arquivo (pode ser CSV ou XLSX)
arquivo = r'C:\Users\Documents\Dados-Sinteticos\Saida_Sinteticos\dataset_peticoes_sintetico_formatado.csv' 

# Leitura do arquivo
df = pd.read_csv(arquivo) # para CSV
# df = pd.read_excel(arquivo) # para Excel

# Função para limpar e alinhar texto
def limpar_texto(texto):
    if pd.isnull(texto):
        return ''
    texto = str(texto)
    texto = texto.strip()  # remove espaços no início/fim
    texto = ' '.join(texto.split())  # remove espaços duplicados dentro do texto
    return texto

# Aplicar a função em todas as células do dataframe
df_limpo = df.applymap(limpar_texto)

# Salvar o dataframe limpo
df_limpo.to_csv(r'C:\Users\Documents\Dados-Sinteticos\Saida_Sinteticos\arquivo_formatado.csv', index=False)  # salvar CSV
# df_limpo.to_excel('arquivo_formatado.xlsx', index=False)  # salvar Excel

print('Arquivo formatado e salvo com sucesso!')

C:\Users\Willgnner\AppData\Local\Temp\ipykernel_15016\3622838125.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpo = df.applymap(limpar_texto)


Arquivo formatado e salvo com sucesso!
